# Импорты библиотек

In [1]:
import sympy
import plotly.express as px
import math

# Входные данные

In [2]:
x = sympy.symbols('x')

n = 10
f = x ** 2 - sympy.log(x)
a = 0.5
b = 1.0
k = 1
m = 10

# Определение $x_i$ и $f_i$

In [3]:
h = (b - a) / n
x_points = []
y_points = []
for i in range(n + 1):
    x_points.append(a + i * h)
    y_points.append(float(f.subs(x, x_points[i])))

figure = px.line(x=x_points, y=y_points, markers=True)
figure.update_traces(name="f(x)", showlegend=True)
figure.add_scatter(x=x_points, y=[float(sympy.diff(f, x, k).subs(x, c)) for c in x_points], name="f'(x)")
figure.show()

# Численное вычисление дифференциала

Реализация функции Лагранжа для $(n + 1)$ узлов:</br>
$\displaystyle L_n(x) = \sum_{i = 0}^{n} f(x_i) \prod_{j = 0; j \neq i}^{n} \frac{x - x_j}{x_i - x_j} = \sum_{i = 0}^{n} f(x_i) \prod_{j = 0; j \neq i}^{n} \frac{x - x_j}{(i - j) h}$

In [4]:
def lagrange_interpolation(n: int) -> sympy.core.add.Add:
    lagrange_polynomial = 0
    for i in range(n + 1):
        multiplication = 1

        for j in range(n + 1):
            if j != i:
                multiplication *= (x - x_points[j]) / ((i - j) * h)

        lagrange_polynomial += y_points[i] * multiplication

    return lagrange_polynomial

$\displaystyle L_n^{(k)}(x) = \left( \sum_{i = 0}^{n} f(x_i) \prod_{j = 0; j \neq i}^{n} \frac{x - x_j}{(i - j) h} \right)^{(k)} = \sum_{i = 0}^{n} f(x_i) \left( \prod_{j = 0; j \neq i}^{n} \frac{x - x_j}{(i - j) h} \right)^{(k)}$

In [5]:
L_nk = sympy.diff(lagrange_interpolation(n), x, k)
L_nk

-18.8629436111989*(2.0 - 2.0*x)*(2.11111111111111 - 2.22222222222222*x)*(2.25 - 2.5*x)*(2.42857142857143 - 2.85714285714286*x)*(2.66666666666667 - 3.33333333333333*x)*(3.0 - 4.0*x)*(3.5 - 5.0*x)*(4.33333333333333 - 6.66666666666667*x)*(6.0 - 10.0*x) - 9.43147180559945*(2.0 - 2.0*x)*(2.11111111111111 - 2.22222222222222*x)*(2.25 - 2.5*x)*(2.42857142857143 - 2.85714285714286*x)*(2.66666666666667 - 3.33333333333333*x)*(3.0 - 4.0*x)*(3.5 - 5.0*x)*(4.33333333333333 - 6.66666666666667*x)*(11.0 - 20.0*x) - 6.28764787039963*(2.0 - 2.0*x)*(2.11111111111111 - 2.22222222222222*x)*(2.25 - 2.5*x)*(2.42857142857143 - 2.85714285714286*x)*(2.66666666666667 - 3.33333333333333*x)*(3.0 - 4.0*x)*(3.5 - 5.0*x)*(6.0 - 10.0*x)*(11.0 - 20.0*x) - 4.71573590279973*(2.0 - 2.0*x)*(2.11111111111111 - 2.22222222222222*x)*(2.25 - 2.5*x)*(2.42857142857143 - 2.85714285714286*x)*(2.66666666666667 - 3.33333333333333*x)*(3.0 - 4.0*x)*(4.33333333333333 - 6.66666666666667*x)*(6.0 - 10.0*x)*(11.0 - 20.0*x) - 3.77258872223978

In [6]:
figure = px.line(x=x_points, y=[float(sympy.diff(f, x, k).subs(x, c)) for c in x_points], markers=True)
figure.update_traces(name="f'(x)", showlegend=True)

figure.add_scatter(x=x_points, y=[float(L_nk.subs(x, c)) for c in x_points], name="L_nk")
figure.show()

# Вычисление $R_{n,k}(x)$ и определение минимума и максимума на отрезке $[a; b]$

Формула остаточного члена при дифференцировании:</br>
$\displaystyle R_{n,k}(x) = \sum_{i = 0}^{k} \frac{f^{(n + i + 1)}(\xi_i)}{(n + i + 1)!} \omega_{n + 1}^{(k - i)}(x),\,\, \xi_i \in [a; b], i = \overline{0,k}$

In [7]:
def remainder_term(n: int, k: int) -> dict:
    remainder_min = 0
    remainder_max = 0

    factorial = math.factorial(n)
    omega_n_plus_1 = 1
    for i in range(n + 1):
        omega_n_plus_1 *= (x - x_points[i])

    for i in range(k + 1):
        factorial *= (n + i + 1)

        f_n_plus_i_plus_1 = sympy.diff(f, x, n + i + 1)
        f_n_plus_i_plus_2 = sympy.diff(f_n_plus_i_plus_1, x)
        extremes_f_n_plus_i_plus_1 = sympy.solve(sympy.Eq(f_n_plus_i_plus_2, 0), x)
        x_f_n_plus_i_plus_1 = x_points + [float(c) for c in extremes_f_n_plus_i_plus_1 if c.is_real and a <= c <= b]
        y_f_n_plus_i_plus_1 = [float(f_n_plus_i_plus_1.subs(x, c)) for c in x_f_n_plus_i_plus_1]
        f_n_plus_i_plus_1_min = min(y_f_n_plus_i_plus_1)
        f_n_plus_i_plus_1_max = max(y_f_n_plus_i_plus_1)

        omega_n_plus_1_k_minus_i = sympy.diff(omega_n_plus_1, x, k - i)
        omega_n_plus_1_k_minus_i_plus_1 = sympy.diff(omega_n_plus_1_k_minus_i, x)
        extremes_omega_n_plus_1_k_minus_i = sympy.solve(sympy.Eq(omega_n_plus_1_k_minus_i_plus_1, 0), x)
        x_omega_n_plus_1_k_minus_i = x_points + [float(c) for c in extremes_omega_n_plus_1_k_minus_i if c.is_real and a <= c <= b]
        y_omega_n_plus_1_k_minus_i = [float(omega_n_plus_1_k_minus_i.subs(x, c)) for c in x_omega_n_plus_1_k_minus_i]
        omega_n_plus_1_k_minus_i_min = min(y_omega_n_plus_1_k_minus_i)
        omega_n_plus_1_k_minus_i_max = max(y_omega_n_plus_1_k_minus_i)

        remainders = [
            f_n_plus_i_plus_1_min * omega_n_plus_1_k_minus_i_min,
            f_n_plus_i_plus_1_min * omega_n_plus_1_k_minus_i_max,
            f_n_plus_i_plus_1_max * omega_n_plus_1_k_minus_i_min,
            f_n_plus_i_plus_1_max * omega_n_plus_1_k_minus_i_max
        ]
        remainder_min += min(remainders) / factorial
        remainder_max += max(remainders) / factorial

    return {
        "min": remainder_min,
        "max": remainder_max
    }

In [8]:
R_nk = remainder_term(n, k)
print("R_nk_min =", R_nk["min"])
print("R_nk_max =", R_nk["max"])

R_nk_min = -6.667253923533049e-05
R_nk_max = 1.481584596973588e-05


# Вычисление $R_{n,k}(x_m)$ и проверка выполнения условий

In [9]:
R_nkm = L_nk.subs(x, x_points[m]) - sympy.diff(f, x, k).subs(x, x_points[m])
print("R_nk(x_m) =", R_nkm)

print(R_nk["min"], R_nkm, R_nk["max"])
print(R_nk["min"] < R_nkm < R_nk["max"])

R_nk(x_m) = 7.24139562979786e-7
-6.667253923533049e-05 7.24139562979786e-7 1.481584596973588e-05
True
